<a href="https://colab.research.google.com/github/weso500/QMLIIOT/blob/main/DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 7.9 MB/s eta 0:00:00


In [2]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error
from matplotlib import pyplot
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import zipfile
import pandas as pd

In [7]:
# Path to your ZIP file
zip_path = '/content/drive/MyDrive/FDA Journal/University of Ottawa Rolling-element Dataset – Vibration and Acoustic Faults under Constant Load and Speed conditions (UORED-VAFCLS).zip'

# Create a dictionary to store DataFrames
dataframes = {}

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as z:
    # Loop through each file in the ZIP
    for filename in z.namelist():
        if filename.endswith('.csv'):
            # Read each CSV into a DataFrame
            with z.open(filename) as f:
                df = pd.read_csv(f)
                dataframes[filename] = df

# Create an empty list to store healthy dataframes
healthy_dfs = []

# Iterate through the dataframes dictionary
for filename, df in dataframes.items():
    # Check if 'Healthy' is in the filename
    if 'Healthy' in filename:
        healthy_dfs.append(df)

# Concatenate the healthy dataframes into a single dataframe
healthy_merged_df = pd.concat(healthy_dfs, ignore_index=True)


# Drop the 'Speed' and 'Load' columns
healthy_merged_df = healthy_merged_df.drop(['Speed', 'Load'], axis=1)

# Display the first few rows of the modified dataframe
display(healthy_merged_df.head())


,Accelerometer,Acoustic,Temperature Difference
0,4.454746,-0.004864,3.760168
1,-0.638000,-0.004206,3.434396
2,-1.443815,-0.003877,4.255864
3,-2.797583,-0.003549,3.435602
4,-3.281071,-0.002562,3.107306


In [4]:
def create_windows(df, window_size, overlap):
    windows = []
    step_size = window_size
    for i in range(0, len(df) - window_size + 1, step_size):
        window = df.iloc[i:i + window_size]
        windows.append(window)
    return windows

window_size = 4200
overlap = int(window_size * 0.5)

healthy_windows = create_windows(healthy_merged_df, window_size, overlap)

# Create empty dictionaries to store data for each sensor
sensor_data = {
    'Accelerometer': [],
    'Acoustic': [],
    'Temperature Difference': []
}

# Iterate through each window
for window in healthy_windows:
    # Extract data for each sensor and append to the respective list
    sensor_data['Accelerometer'].append(window['Accelerometer'].values)
    sensor_data['Acoustic'].append(window['Acoustic'].values)
    sensor_data['Temperature Difference'].append(window['Temperature Difference'].values)

# Convert the lists of arrays into dataframes
accelerometer_df = pd.DataFrame(sensor_data['Accelerometer'])
acoustic_df = pd.DataFrame(sensor_data['Acoustic'])
temperature_df = pd.DataFrame(sensor_data['Temperature Difference'])

# Display the shapes of the new dataframes to confirm
print(f"Shape of Accelerometer dataframe: {accelerometer_df.shape}")
print(f"Shape of Acoustic dataframe: {acoustic_df.shape}")
print(f"Shape of Temperature Difference dataframe: {temperature_df.shape}")

Shape of Accelerometer dataframe: (2000, 4200)
Shape of Acoustic dataframe: (2000, 4200)
Shape of Temperature Difference dataframe: (2000, 4200)


In [9]:
# Path to your ZIP file
zip_path = '/content/drive/MyDrive/FDA Journal/University of Ottawa Rolling-element Dataset – Vibration and Acoustic Faults under Constant Load and Speed conditions (UORED-VAFCLS).zip'

# Create a dictionary to store DataFrames
fault_dataframes = {}

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as z:
    # Loop through each file in the ZIP
    for filename in z.namelist():
        if filename.endswith('.csv') and 'Fault' in filename:
            # Read each CSV into a DataFrame
            with z.open(filename) as f:
                df = pd.read_csv(f)
                fault_dataframes[filename] = df

# Create empty lists to store dataframes for developing and faulty conditions
fault_developing_dfs = []
faulty_dfs = []

# Iterate through the fault_dataframes dictionary
for filename, df in fault_dataframes.items():
    # Extract the fault condition from the filename using the labeling scheme
    # Assuming the format is Type_Bearing_Condition.csv (e.g., I_1_1.csv)
    parts = filename.split('/')[-1].split('.')[0].split('_')
    if len(parts) >= 3:
        condition = parts[2]
        if condition == '1':
            fault_developing_dfs.append(df)
        elif condition == '2':
            faulty_dfs.append(df)

# Concatenate the dataframes for each condition
fault_developing_merged_df = pd.concat(fault_developing_dfs, ignore_index=True)
faulty_merged_df = pd.concat(faulty_dfs, ignore_index=True)

# Display the first few rows of the merged dataframes
print("Fault Developing Merged DataFrame:")
display(fault_developing_merged_df.head())

print("\nFaulty Merged DataFrame:")
display(faulty_merged_df.head())


# Drop the 'Speed' and 'Load' columns from fault dataframes
fault_developing_merged_df = fault_developing_merged_df.drop(['Speed', 'Load'], axis=1)
faulty_merged_df = faulty_merged_df.drop(['Speed', 'Load'], axis=1)

# Display the first few rows of the modified fault dataframes
print("Fault Developing Merged DataFrame after normalization and dropping columns:")
display(fault_developing_merged_df.head())

print("\nFaulty Merged DataFrame after normalization and dropping columns:")
display(faulty_merged_df.head())


# Apply windowing to the fault developing and faulty dataframes
fault_developing_windows = create_windows(fault_developing_merged_df, window_size, overlap)
faulty_windows = create_windows(faulty_merged_df, window_size, overlap)

# Create empty dictionaries to store data for each sensor
sensor_data_faulty = {
    'Accelerometer': [],
    'Acoustic': [],
    'Temperature Difference': []
}

# Iterate through each window
for window in fault_developing_windows:
    # Extract data for each sensor and append to the respective list
    sensor_data_faulty['Accelerometer'].append(window['Accelerometer'].values)
    sensor_data_faulty['Acoustic'].append(window['Acoustic'].values)
    sensor_data_faulty['Temperature Difference'].append(window['Temperature Difference'].values)

# Convert the lists of arrays into dataframes
accelerometer_df_faulty = pd.DataFrame(sensor_data_faulty['Accelerometer'])
acoustic_df_faulty = pd.DataFrame(sensor_data_faulty['Acoustic'])
temperature_df_faulty = pd.DataFrame(sensor_data_faulty['Temperature Difference'])

# Display the shapes of the new dataframes to confirm
print(f"Shape of Accelerometer dataframe: {accelerometer_df_faulty.shape}")
print(f"Shape of Acoustic dataframe: {acoustic_df_faulty.shape}")
print(f"Shape of Temperature Difference dataframe: {temperature_df_faulty.shape}")

Fault Developing Merged DataFrame:


,Accelerometer,Acoustic,Speed,Load,Temperature Difference
0,19.700756,0.074370,1819,400,22.575598
1,19.829687,0.083576,0,0,21.964033
2,22.021502,0.088179,0,0,22.583118
3,23.923225,0.088508,0,0,21.308137
4,24.148853,0.092453,0,0,22.106005



Faulty Merged DataFrame:


,Accelerometer,Acoustic,Speed,Load,Temperature Difference
0,34.785607,-0.025905,1819,400,24.261550
1,35.591422,-0.024261,0,0,24.221602
2,37.815471,-0.019987,0,0,25.064400
3,36.784028,-0.018672,0,0,24.425321
4,40.942032,-0.021302,0,0,26.213233


Fault Developing Merged DataFrame after normalization and dropping columns:


,Accelerometer,Acoustic,Temperature Difference
0,19.700756,0.074370,22.575598
1,19.829687,0.083576,21.964033
2,22.021502,0.088179,22.583118
3,23.923225,0.088508,21.308137
4,24.148853,0.092453,22.106005



Faulty Merged DataFrame after normalization and dropping columns:


,Accelerometer,Acoustic,Temperature Difference
0,34.785607,-0.025905,24.261550
1,35.591422,-0.024261,24.221602
2,37.815471,-0.019987,25.064400
3,36.784028,-0.018672,24.425321
4,40.942032,-0.021302,26.213233


Shape of Accelerometer dataframe: (2000, 4200)
Shape of Acoustic dataframe: (2000, 4200)
Shape of Temperature Difference dataframe: (2000, 4200)


In [13]:
from scipy.stats import kurtosis, skew
from scipy.fft import fft
import numpy as np

def calculate_features(windows, sensor_name):
    features_list = []
    for window in windows:
        # Calculate RMS
        rms = np.sqrt(np.mean(window**2))

        # Calculate Kurtosis and Skewness
        kurt = kurtosis(window)
        sk = skew(window)

        # Calculate FFT and energy bands (example: 4 bands)
        yf = fft(window)
        n = len(window)
        frequencies = np.fft.fftfreq(n)
        # Get the positive frequencies and their corresponding FFT values
        positive_frequencies = frequencies[:n//2]
        positive_yf = np.abs(yf[:n//2])

        # Define frequency bands (adjust as needed based on your data's frequency range)
        # For example, let's divide the positive frequencies into 4 equal bands
        band_width = len(positive_frequencies) // 4
        fft_bands = []
        for i in range(4):
            start = i * band_width
            end = (i + 1) * band_width if i < 3 else len(positive_frequencies)
            band_energy = np.sum(positive_yf[start:end]**2)
            fft_bands.append(band_energy)


        # Calculate Entropy (example: Spectral Entropy)
        # Normalize the power spectrum to get probability distribution
        power_spectrum = positive_yf**2
        total_energy = np.sum(power_spectrum)
        if total_energy > 0:
            probability_distribution = power_spectrum / total_energy
            # Avoid log(0) by replacing zeros with a small epsilon
            epsilon = 1e-10
            probability_distribution[probability_distribution == 0] = epsilon
            entropy = -np.sum(probability_distribution * np.log2(probability_distribution))
        else:
            entropy = 0 # Handle cases with zero total energy

        features = {
            f'{sensor_name}_RMS': rms,
            f'{sensor_name}_Kurtosis': kurt,
            f'{sensor_name}_Skewness': sk,
            f'{sensor_name}_Entropy': entropy
        }
        # Add FFT bands to the features dictionary
        for i, band_energy in enumerate(fft_bands):
            features[f'{sensor_name}_FFT_Band_{i+1}'] = band_energy

        features_list.append(features)

    return pd.DataFrame(features_list)

# Calculate features for each sensor
accelerometer_features_df = calculate_features(sensor_data['Accelerometer'], 'Accelerometer')
acoustic_features_df = calculate_features(sensor_data['Acoustic'], 'Acoustic')
temperature_features_df = calculate_features(sensor_data['Temperature Difference'], 'Temperature')
accelerometer_features_faulty = calculate_features(sensor_data_faulty['Accelerometer'], 'Accelerometer')
acoustic_features_faulty = calculate_features(sensor_data_faulty['Acoustic'], 'Acoustic')
temperature_features_faulty = calculate_features(sensor_data_faulty['Temperature Difference'], 'Temperature')

# Display the first few rows of the new dataframes
print("Accelerometer Features DataFrame:")
display(accelerometer_features_df.head())
print(accelerometer_features_df.shape)
print(accelerometer_features_faulty.shape)

print("\nAcoustic Features DataFrame:")
display(acoustic_features_df.head())

print("\nTemperature Features DataFrame:")
display(temperature_features_df.head())

Accelerometer Features DataFrame:


,Accelerometer_RMS,Accelerometer_Kurtosis,Accelerometer_Skewness,Accelerometer_Entropy,Accelerometer_FFT_Band_1,Accelerometer_FFT_Band_2,Accelerometer_FFT_Band_3,Accelerometer_FFT_Band_4
0,0.473201,1.946419,-0.013277,0.002477,3.949451e+06,7.657936,26.869585,8.191180
1,0.473305,1.042136,0.061701,0.002777,3.951131e+06,8.650118,23.278551,8.291966
2,0.473291,0.580422,-0.082945,0.002610,3.950919e+06,9.572151,25.939991,7.903715
3,0.473328,-0.020088,-0.038325,0.002304,3.951599e+06,7.691318,23.073395,10.398373
4,0.473344,0.224499,0.067521,0.002116,3.951908e+06,8.547743,27.991764,9.791571


(2000, 8)
(2000, 8)

Acoustic Features DataFrame:


,Acoustic_RMS,Acoustic_Kurtosis,Acoustic_Skewness,Acoustic_Entropy,Acoustic_FFT_Band_1,Acoustic_FFT_Band_2,Acoustic_FFT_Band_3,Acoustic_FFT_Band_4
0,0.057482,-0.347284,0.022972,0.011220,58243.843345,0.113917,0.088986,0.077397
1,0.055116,-0.535199,-0.045974,0.010863,53549.304852,0.119659,0.089692,0.086658
2,0.056425,-0.148005,-0.209297,0.008794,56133.259570,0.126323,0.084639,0.075348
3,0.055826,-0.406852,-0.128707,0.010626,54939.015276,0.112280,0.084869,0.075235
4,0.057450,-0.511392,-0.077867,0.008429,58191.046716,0.122727,0.086633,0.077708



Temperature Features DataFrame:


,Temperature_RMS,Temperature_Kurtosis,Temperature_Skewness,Temperature_Entropy,Temperature_FFT_Band_1,Temperature_FFT_Band_2,Temperature_FFT_Band_3,Temperature_FFT_Band_4
0,0.531366,0.034605,0.019206,0.003809,4.979319e+06,174.239024,185.545997,178.161055
1,0.531653,0.025577,0.026567,0.003835,4.984702e+06,193.383064,173.679552,162.977851
2,0.531762,-0.117318,-0.026683,0.003970,4.986686e+06,214.856222,175.679410,163.783276
3,0.531827,0.136532,-0.074479,0.003855,4.987975e+06,193.087649,153.480641,166.899126
4,0.532003,0.051491,-0.068103,0.003769,4.991270e+06,184.212544,183.929231,179.957293


In [14]:
# Merge the feature dataframes
merged_features_df = pd.concat([accelerometer_features_df, acoustic_features_df, temperature_features_df], axis=1)
merged_features_faulty = pd.concat([accelerometer_features_faulty, acoustic_features_faulty, temperature_features_faulty], axis=1)
# Display the first few rows and the shape of the merged dataframe to confirm
print("Merged Features DataFrame:")
display(merged_features_df.head())
print(f"\nShape of Merged Features dataframe: {merged_features_df.shape}")
print(f"\nShape of Merged Features faulty dataframe: {merged_features_faulty.shape}")

Merged Features DataFrame:


,Accelerometer_RMS,Accelerometer_Kurtosis,Accelerometer_Skewness,Accelerometer_Entropy,Accelerometer_FFT_Band_1,Accelerometer_FFT_Band_2,Accelerometer_FFT_Band_3,Accelerometer_FFT_Band_4,Acoustic_RMS,Acoustic_Kurtosis,...,Acoustic_FFT_Band_3,Acoustic_FFT_Band_4,Temperature_RMS,Temperature_Kurtosis,Temperature_Skewness,Temperature_Entropy,Temperature_FFT_Band_1,Temperature_FFT_Band_2,Temperature_FFT_Band_3,Temperature_FFT_Band_4
0,0.473201,1.946419,-0.013277,0.002477,3.949451e+06,7.657936,26.869585,8.191180,0.057482,-0.347284,...,0.088986,0.077397,0.531366,0.034605,0.019206,0.003809,4.979319e+06,174.239024,185.545997,178.161055
1,0.473305,1.042136,0.061701,0.002777,3.951131e+06,8.650118,23.278551,8.291966,0.055116,-0.535199,...,0.089692,0.086658,0.531653,0.025577,0.026567,0.003835,4.984702e+06,193.383064,173.679552,162.977851
2,0.473291,0.580422,-0.082945,0.002610,3.950919e+06,9.572151,25.939991,7.903715,0.056425,-0.148005,...,0.084639,0.075348,0.531762,-0.117318,-0.026683,0.003970,4.986686e+06,214.856222,175.679410,163.783276
3,0.473328,-0.020088,-0.038325,0.002304,3.951599e+06,7.691318,23.073395,10.398373,0.055826,-0.406852,...,0.084869,0.075235,0.531827,0.136532,-0.074479,0.003855,4.987975e+06,193.087649,153.480641,166.899126
4,0.473344,0.224499,0.067521,0.002116,3.951908e+06,8.547743,27.991764,9.791571,0.057450,-0.511392,...,0.086633,0.077708,0.532003,0.051491,-0.068103,0.003769,4.991270e+06,184.212544,183.929231,179.957293



Shape of Merged Features dataframe: (2000, 24)

Shape of Merged Features faulty dataframe: (2000, 24)


In [15]:
# Take 500 rows from the healthy features dataframe
healthy_test_df = merged_features_df.sample(n=500, random_state=42)

# Remove the sampled rows from the original healthy features dataframe
merged_features_df = merged_features_df.drop(healthy_test_df.index)


# Take 10 rows from the faulty features dataframe
faulty_test_df = merged_features_faulty.sample(n=10, random_state=42)

# Assign target labels
healthy_test_df['target'] = 0
faulty_test_df['target'] = 1

# Concatenate the two dataframes to create the test dataset
test_df = pd.concat([healthy_test_df, faulty_test_df], ignore_index=True)

# Display the first few rows and the shape of the test dataset
print("Test Dataset:")
display(test_df.head())
print(f"\nShape of Test Dataset: {test_df.shape}")

# Display the shape of the modified healthy features dataframe to confirm
print(f"\nShape of modified Merged Features dataframe: {merged_features_df.shape}")

Test Dataset:


,Accelerometer_RMS,Accelerometer_Kurtosis,Accelerometer_Skewness,Accelerometer_Entropy,Accelerometer_FFT_Band_1,Accelerometer_FFT_Band_2,Accelerometer_FFT_Band_3,Accelerometer_FFT_Band_4,Acoustic_RMS,Acoustic_Kurtosis,...,Acoustic_FFT_Band_4,Temperature_RMS,Temperature_Kurtosis,Temperature_Skewness,Temperature_Entropy,Temperature_FFT_Band_1,Temperature_FFT_Band_2,Temperature_FFT_Band_3,Temperature_FFT_Band_4,target
0,0.479039,-0.098615,-0.058582,0.011101,4.045585e+06,20.801458,48.257813,17.430620,0.053515,0.566243,...,0.088827,0.547039,0.010839,-0.038738,0.003958,5.277289e+06,202.229655,235.402253,200.602506,0
1,0.599658,5.278540,-0.684906,0.005580,6.341291e+06,12.412551,0.229111,0.336639,0.066706,-0.854790,...,0.058800,0.501582,-0.231688,-0.005793,0.005270,4.436205e+06,214.448212,305.627559,220.720356,0
2,0.472997,-0.077290,-0.010790,0.012103,3.943936e+06,23.179530,30.971245,30.452768,0.054537,-0.280030,...,0.201745,0.528891,-0.015894,0.020419,0.003763,4.933043e+06,182.747132,203.266616,172.862201,0
3,0.476118,0.659216,0.044906,0.000940,3.998598e+06,7.208924,19.881202,1.948268,0.055040,0.572461,...,0.293551,0.557433,-0.168775,0.005038,0.004659,5.479401e+06,234.815873,349.263268,226.224665,0
4,0.475679,0.456724,0.056674,0.015705,3.987825e+06,16.309029,46.225939,23.539537,0.053165,-0.197097,...,0.149612,0.493222,-0.145222,-0.038806,0.004945,4.289681e+06,202.289769,259.904379,193.727494,0



Shape of Test Dataset: (510, 25)

Shape of modified Merged Features dataframe: (1500, 24)


In [16]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data (merged_features_df) and transform it
merged_features_scaled = scaler.fit_transform(merged_features_df)

# Transform the test data (test_df) using the scaler fitted on the training data
test_df_scaled = scaler.transform(test_df.drop('target', axis=1))

# Create new DataFrames for the scaled data
merged_features_scaled_df = pd.DataFrame(merged_features_scaled, columns=merged_features_df.columns)
test_df_scaled_df = pd.DataFrame(test_df_scaled, columns=test_df.drop('target', axis=1).columns)

# Display the first few rows of the scaled dataframes to confirm
print("Scaled Merged Features DataFrame:")
display(merged_features_scaled_df.head())

print("\nScaled Test DataFrame:")
display(test_df_scaled_df.head())

Scaled Merged Features DataFrame:


,Accelerometer_RMS,Accelerometer_Kurtosis,Accelerometer_Skewness,Accelerometer_Entropy,Accelerometer_FFT_Band_1,Accelerometer_FFT_Band_2,Accelerometer_FFT_Band_3,Accelerometer_FFT_Band_4,Acoustic_RMS,Acoustic_Kurtosis,...,Acoustic_FFT_Band_3,Acoustic_FFT_Band_4,Temperature_RMS,Temperature_Kurtosis,Temperature_Skewness,Temperature_Entropy,Temperature_FFT_Band_1,Temperature_FFT_Band_2,Temperature_FFT_Band_3,Temperature_FFT_Band_4
0,-0.529888,1.617617,-0.055681,-0.284277,-0.509814,-0.298194,-0.087415,-0.249321,-0.276943,-0.458099,...,-0.623747,-0.570255,0.044571,1.120655,1.089206,-0.233756,0.021503,-0.380768,-0.652744,-0.482840
1,-0.529012,0.654387,0.486554,-0.277257,-0.509190,-0.281108,-0.156900,-0.244942,-0.290120,-0.735747,...,-0.621671,-0.500784,0.053825,1.041083,1.288442,-0.233202,0.030594,-0.251032,-0.729395,-0.698509
2,-0.528824,-0.477081,-0.236830,-0.288346,-0.509016,-0.297619,-0.160870,-0.153426,-0.286163,-0.546112,...,-0.635842,-0.586475,0.059433,2.019054,-1.446428,-0.232798,0.036122,-0.253034,-0.859871,-0.642809
3,-0.528686,-0.216551,0.528644,-0.292744,-0.508901,-0.282871,-0.065701,-0.179789,-0.277119,-0.700573,...,-0.630657,-0.567924,0.065123,1.269492,-1.273863,-0.234581,0.041687,-0.313179,-0.663187,-0.457325
4,-0.528502,0.291509,-0.171820,-0.278446,-0.508808,-0.278309,-0.098960,-0.262860,-0.281002,-0.582782,...,-0.628652,-0.621365,0.057568,-0.521994,0.822250,-0.228124,0.034018,-0.349902,-0.295485,-0.330776



Scaled Test DataFrame:


,Accelerometer_RMS,Accelerometer_Kurtosis,Accelerometer_Skewness,Accelerometer_Entropy,Accelerometer_FFT_Band_1,Accelerometer_FFT_Band_2,Accelerometer_FFT_Band_3,Accelerometer_FFT_Band_4,Acoustic_RMS,Acoustic_Kurtosis,...,Acoustic_FFT_Band_3,Acoustic_FFT_Band_4,Temperature_RMS,Temperature_Kurtosis,Temperature_Skewness,Temperature_Entropy,Temperature_FFT_Band_1,Temperature_FFT_Band_2,Temperature_FFT_Band_3,Temperature_FFT_Band_4
0,-0.480898,-0.560727,-0.383328,-0.082182,-0.474091,-0.071858,0.326439,0.152100,-0.299040,0.891665,...,-0.232544,-0.484516,0.550124,0.911184,-0.479092,-0.230678,0.524766,-0.191080,-0.330696,-0.164071
1,0.531176,5.166951,-4.912874,-0.211556,0.379004,-0.216318,-0.602899,-0.590573,-0.225561,-1.207954,...,-0.664559,-0.709754,-0.916110,-1.226483,0.412593,-0.203611,-0.895803,-0.108277,0.122925,0.121692
2,-0.531598,-0.538012,-0.037700,-0.058719,-0.511864,-0.030907,-0.008050,0.717867,-0.293343,-0.358728,...,0.028760,0.362506,-0.035244,0.675550,1.122045,-0.234690,-0.056657,-0.323110,-0.538277,-0.558107
3,-0.505410,0.246504,0.365094,-0.320307,-0.491551,-0.305926,-0.222638,-0.520553,-0.290541,0.900853,...,1.219210,1.051162,0.885377,-0.671959,0.705747,-0.216221,0.866128,0.029752,0.404791,0.199878
4,-0.509093,0.030814,0.450198,0.025708,-0.495554,-0.149219,0.287123,0.417511,-0.300986,-0.236192,...,-0.190562,-0.028551,-1.185766,-0.464358,-0.480928,-0.210306,-1.143279,-0.190673,-0.172424,-0.261727


In [23]:
from sklearn.decomposition import PCA

# Initialize PCA with 3 components
pca = PCA(n_components=8)

# Fit PCA to the merged features and transform the data
pca_features = pca.fit_transform(merged_features_scaled_df)

# Create a new DataFrame for the PCA features
pca_features_df = pd.DataFrame(pca_features, columns=[f'PCA_Feature_{i+1}' for i in range(8)])

# Calculate the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

print("Explained Variance Ratio for each component:", explained_variance_ratio)
print("Total Explained Variance:", sum(explained_variance_ratio))

# Display the first few rows of the PCA features dataframe
print("\nPCA Features DataFrame:")
display(pca_features_df.head())

Explained Variance Ratio for each component: [0.3170483  0.17688679 0.13603891 0.07347807 0.06652839 0.05766864
 0.04130897 0.03418989]
Total Explained Variance: 0.9031479563419379

PCA Features DataFrame:


,PCA_Feature_1,PCA_Feature_2,PCA_Feature_3,PCA_Feature_4,PCA_Feature_5,PCA_Feature_6,PCA_Feature_7,PCA_Feature_8
0,-0.798351,-1.071834,-0.490544,-0.329769,-1.221815,0.604708,0.747248,-0.907175
1,-0.758427,-1.132681,-0.432647,0.025948,-1.361405,-0.384485,0.997647,-0.926774
2,-0.904675,-1.102351,-0.282038,0.003579,-1.323094,-1.020833,-1.653868,0.191859
3,-0.762137,-1.065979,-0.259230,0.165790,-1.393048,-1.051815,-1.490184,-0.351433
4,-0.777511,-1.254159,-0.565796,0.380128,-1.120004,0.347569,0.613188,-1.007039


In [24]:
pca_features_test = pca.transform(test_df_scaled_df)

# Create a new DataFrame for the PCA features
pca_featurestest_df = pd.DataFrame(pca_features_test, columns=[f'PCA_Feature_{i+1}' for i in range(8)])

In [27]:
# Add the 'target' column back to the pca test dataframe
pca_featurestest_df['target'] = test_df['target']

# Display the first few rows of the modified pca test dataframe to confirm
print("PCA Test DataFrame with Target:")
display(pca_featurestest_df.head())
print(f"\nShape of PCA Test Dataset with Target: {pca_featurestest_df.shape}")

PCA Test DataFrame with Target:


,PCA_Feature_1,PCA_Feature_2,PCA_Feature_3,PCA_Feature_4,PCA_Feature_5,PCA_Feature_6,PCA_Feature_7,PCA_Feature_8,target
0,0.245098,-0.454371,-0.625969,0.542562,0.040275,-0.378847,-0.713943,-0.055827,0
1,-1.912014,-0.870226,-0.176848,-2.359543,-1.310005,6.698016,0.645263,0.893167,0
2,0.833747,-0.551962,-0.461520,-0.334091,0.110693,-0.694896,1.085830,0.138136,0
3,1.496393,0.169967,-1.580089,0.328049,1.890110,0.173592,0.948131,1.478359,0
4,0.204263,-0.639307,-0.105082,-1.500889,-0.781131,-0.600246,-0.344658,-0.279620,0



Shape of PCA Test Dataset with Target: (510, 9)


In [28]:
# Save the pca_featurestest_df as a CSV file
pca_featurestest_df.to_csv('pca_test_features_with_target.csv', index=False)

print("pca_featurestest_df saved to 'pca_test_features_with_target.csv'")

pca_featurestest_df saved to 'pca_test_features_with_target.csv'


In [29]:
# Save the pca_features_df as a CSV file
pca_features_df.to_csv('pca_train_features.csv', index=False)

print("pca_features_df saved to 'pca_train_features.csv'")

pca_features_df saved to 'pca_train_features.csv'
